# HOUMIL: High Utility Occupancy Pattern Mining with Indexed List.
This notebook implements the HOUMIL algorithm for for Mining High Utility Occupancy Patterns Based on Indexed List Structure

<details>
<summary><strong>Table of Contents</strong></summary>

1. [Imports and Setup](#imports-and-setup)
   Libraries and shared utilities.
2. [QuadrupleEntry Class](#quadrupleentry-class)
   Indexed-list entry structure.
3. [IndexedList Class](#indexedlist-class)
   Global/conditional indexed lists.
4. [HUOMIL Algorithm Class](#huomil-algorithm-class)
   Main mining algorithm.
5. [Example Run (Toy Data)](#example-run-toy-data)
   Sanity check run.
</details>


## Imports and Setup


In [1]:
import os
import time
import random
import tracemalloc
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Tuple, Set
import heapq
import matplotlib.pyplot as plt
import os
import glob
import seaborn as sns

## QuadrupleEntry Class


In [2]:
# QuadrupleEntry Class
class QuadrupleEntry:
    """
    Data structure for storing transaction item relationships in HUOMIL.
    
    PURPOSE:
    Represents a single entry in an indexed list, linking items within a 
    transaction through index pointers instead of transaction IDs.
    
    ATTRIBUTES:
    next_item : str or None
        The next item that appears after this item in the transaction.
        None if this is the last item.
    
    uo : float
        Utility Occupancy - the ratio of this item's utility to total 
        transaction utility (u(i,T) / TU(T)).
        Range: [0.0, 1.0]
    
    ruo : float
        Remaining Utility Occupancy - sum of utility occupancies for all 
        items that appear AFTER this item in the transaction.
        Range: [0.0, 1.0]
    
    next_idx : int or None
        Index pointer to the next_item's entry in its GUO-IL.
        Enables O(1) access without TID comparison (key innovation).
        None if next_item is None.
    
    EXAMPLE:
    Transaction T1: {(A, qty=2), (B, qty=3), (C, qty=1)}
    External utilities: {A: 5, B: 2, C: 3}
    
    Transaction Utility (TU): (2×5) + (3×2) + (1×3) = 19
    
    Processing in REVERSE order (C → B → A):
    
    Entry for C:
        QuadrupleEntry(
            next_item=None,      # C is last item
            uo=3/19=0.158,       # u(C,T1) / TU(T1)
            ruo=0.0,             # No items after C
            next_idx=None        # No next item
        )
    
    Entry for B:
        QuadrupleEntry(
            next_item='C',       # C comes after B
            uo=6/19=0.316,       # u(B,T1) / TU(T1)
            ruo=0.158,           # uo(C) = 0.158
            next_idx=0           # Pointer to C's entry at index 0
        )
    
    Entry for A:
        QuadrupleEntry(
            next_item='B',       # B comes after A
            uo=10/19=0.526,      # u(A,T1) / TU(T1)
            ruo=0.474,           # uo(B) + uo(C) = 0.316 + 0.158
            next_idx=0           # Pointer to B's entry at index 0
        )
    """
    __slots__ = ['next_item', 'uo', 'ruo', 'next_idx']
    
    def __init__(self, next_item, uo, ruo, next_idx):
        """
        Initialize a QuadrupleEntry.
        
        Parameters
        next_item : str or None
            Next item in transaction sequence
        uo : float
            Utility occupancy value [0.0, 1.0]
        ruo : float
            Remaining utility occupancy [0.0, 1.0]
        next_idx : int or None
            Index pointer to next item's entry
        """
        self.next_item = next_item
        self.uo = uo
        self.ruo = ruo
        self.next_idx = next_idx

print("QuadrupleEntry class loaded (memory-optimized with __slots__)")

QuadrupleEntry class loaded (memory-optimized with __slots__)


## IndexedList Class


In [3]:
# IndexedList Class
class IndexedList:
    """
    GUO-IL (Global Utility Occupancy Indexed List) or CUO-IL (Conditional Utility Occupancy Indexed List)
    
    PURPOSE:
    Core data structure in HUOMIL for storing all QuadrupleEntry objects for a specific 
    item or itemset (pattern). Enables efficient traversal, pruning, and pattern extension.
    
    KEY FEATURES:
    - Stores QuadrupleEntry objects indexed by pattern (single item or tuple itemset)
    - Maintains aggregated sum_uo and sum_ruo for fast pruning decisions
    - Supports O(1) access to next-item relationships via next_idx pointers
    - Memory-optimized using __slots__ to eliminate per-instance __dict__
    
    ATTRIBUTES:
    pattern : tuple of str
        The itemset this IndexedList represents. Always stored as tuple for consistency.
        Single items are converted to 1-tuples during initialization.
    
    entries : list[QuadrupleEntry]
        List of QuadrupleEntry objects for transactions containing this pattern.
        Each entry links to the next item in its original transaction.
    
    sum_uo : float
        Pre-computed sum of uo values across all entries. Used for pruning checks.
    
    sum_ruo : float  
        Pre-computed sum of ruo values across all entries. Used for pruning checks.
    
    USAGE EXAMPLE:
    il_A = IndexedList('A')  # Represents pattern {A}
    il_A.add_entry(QuadrupleEntry(next_item='B', uo=0.526, ruo=0.474, next_idx=0))
    il_A.get_support()  # Returns 1
    il_A.get_uo()       # Returns 0.526
    
    il_AB = IndexedList(('A', 'B'))  # Represents pattern {A,B}
    """
    __slots__ = ['pattern', 'entries', 'sum_uo', 'sum_ruo']
    
    def __init__(self, pattern):
        """
        Initialize an IndexedList for a given pattern.
        
        Parameters:
        pattern : str or tuple
            Single item (str) or itemset (tuple of str) this list represents
        """
        self.pattern = pattern if isinstance(pattern, tuple) else tuple([pattern])
        self.entries = []  # List of QuadrupleEntry
        self.sum_uo = 0.0  # Sum of utility occupancy (aggregated info)
        self.sum_ruo = 0.0  # Sum of remaining utility occupancy
    
    def add_entry(self, entry: QuadrupleEntry):
        """
        Add QuadrupleEntry and update aggregated sums for pruning.
        
        Parameters:
        entry : QuadrupleEntry
            Entry to append to this IndexedList
        """
        """Add entry and update aggregated information"""
        self.entries.append(entry)
        self.sum_uo += entry.uo
        self.sum_ruo += entry.ruo
    
    def get_support(self):
        """
        Return support count (number of transactions containing this pattern).
        
        Returns:
        int: Number of QuadrupleEntry objects (== transaction support)
        """
        """Return support (number of entries)"""
        return len(self.entries)
    
    def get_uo(self):
        """
        Calculate average utility occupancy across all entries.
        
        Returns:
        float: Average uo value, or 0.0 if empty
        """
        """Calculate utility occupancy"""
        if len(self.entries) == 0:
            return 0.0
        return self.sum_uo / len(self.entries)


print("IndexedList class loaded!")

IndexedList class loaded!


## HUOMIL Algorithm Class


In [4]:
# HUOMIL Algorithm Class
class HUOMIL:
    """
    HUOMIL: High Utility Occupancy Pattern Mining with Indexed List Structure
    
    REFERENCE:
    Kim et al. (2023) - IEEE Access
    "Efficient Method for Mining High Utility Occupancy Patterns Based on Indexed List Structure"
    
    PURPOSE:
    Complete implementation of the HUOMIL algorithm for mining high utility occupancy (HUO) patterns.
    Dramatically faster than traditional utility mining by replacing TID-lists with IndexedLists
    containing QuadrupleEntry index pointers (Lemma 2: O(1) next-item access).
    
    IMPLEMENTS:
    - Algorithm 1: HUOMIL main procedure (Lines 1-13)
    - Algorithm 2: ConstructGUOIL procedure (Lines 1-19) 
    - Algorithm 3: Search procedure (Lines 1-32)
    
    CORE INNOVATIONS:
    1. GUO-IL/CUO-IL structures replace expensive TID-list intersections
    2. Reverse transaction scanning builds O(1) index pointers (Algorithm 2)
    3. Two-level pruning: LUBUO (loose) + UBUO (tight) upper bounds
    4. Direct chain following for pattern extension (no TID comparison)
    
    
    USAGE EXAMPLE:
    huomil = HUOMIL(min_sup_ratio=0.001, min_uo_ratio=0.2)
    results = huomil.fit(transactions_dict, profit_table)
    # results: [(('A','B'), 150, 0.25), (('B','C'), 120, 0.18), ...]
    """
    
    def __init__(self, min_sup_ratio: float, min_uo_ratio: float):
        """
        Initialize HUOMIL with minimum thresholds.
        
        PARAMETERS:
        min_sup_ratio : float [0.0, 1.0]
            Minimum support ratio (α). Absolute min_sup = ceil(|D| × α)
        min_uo_ratio : float [0.0, 1.0]  
            Minimum utility occupancy ratio (β). Pattern uo(X) ≥ β
            
        INTERNAL STATE:
        - min_sup: Absolute minimum support (calculated from |D|)
        - transactions_dict: Input database {TID: [(item, qty), ...]}
        - external_utilities: {item: profit_per_unit}
        - guo_ils: {item: IndexedList} global indexed lists
        """
        """Initialize HUOMIL"""
        
        self.min_sup_ratio = min_sup_ratio
        self.min_uo_ratio = min_uo_ratio
        
        # Will be set during fit()
        self.min_sup = 0
        self.transactions_dict = None
        self.external_utilities = None
        
        # Results
        self.result_patterns = []
        
        # Item support for first scan
        self.item_support = defaultdict(int)
        
        # Total order (support-ascending)
        self.total_order = []
        self.item_to_idx = {}
        
        # Global indexed lists
        self.guo_ils = {}
    
    def calculate_utility(self, item: str, quantity: int) -> float:
        """
        Calculate internal utility of item in transaction.
        
        FORMULA:
        u(i,T) = quantity(i,T) × external_utility(i)
        
        PARAMETERS:
        item : str
            Item identifier
        quantity : int
            Quantity in transaction
            
        RETURNS:
        float: Item utility (0.0 if item not in external_utilities)
        """
        """Calculate utility: quantity × external_utility"""
        return quantity * self.external_utilities.get(item, 0)
    
    def calculate_transaction_utility(self, transaction: List[Tuple[str, int]]) -> float:
        """
        Calculate total utility of transaction.
        
        FORMULA:
        TU(T) = Σ_{i∈T} u(i,T)
        
        PARAMETERS:
        transaction : List[Tuple[str,int]]
            List of (item, quantity) tuples
            
        RETURNS:
        float: Total transaction utility
        """
        """Calculate total utility of transaction"""
        total = 0.0
        for item, quantity in transaction:
            total += self.calculate_utility(item, quantity)
        return total
    
    def first_database_scan(self):
        """
        Algorithm 1, Lines 1-10: First scan to calculate item support and total order.
        
        STEPS:
        1. Count support for each item across all transactions
        2. Filter items: support ≥ min_sup  
        3. Sort promising items by support (ascending) → total_order (Def. 7)
        
        MODIFIES:
        - self.item_support: {item: support_count}
        - self.total_order: List[str] sorted by support
        - self.item_to_idx: {item: position_in_total_order}
        """
        """
        Algorithm 1, Lines 1-10: First scan to calculate support
        """
        print("  [1/3] First database scan: calculating item support...")
        
        for tid, transaction in self.transactions_dict.items():
            for item, _ in transaction:
                self.item_support[item] += 1
        
        # Filter items by minimum support and create total order
        promising_items = [(item, sup) for item, sup in self.item_support.items() 
                          if sup >= self.min_sup]
        
        # Sort by support (ascending order) - Definition 7 from paper
        promising_items.sort(key=lambda x: x[1])
        self.total_order = [item for item, _ in promising_items]
        self.item_to_idx = {item: idx for idx, item in enumerate(self.total_order)}
        
        print(f"      Found {len(self.total_order)} promising items (min_sup={self.min_sup})")
    
    def construct_guo_ils(self):
        """
        Algorithm 2: ConstructGUOIL procedure (Lines 1-19).
        
        KEY INNOVATION:
        Process transactions in REVERSE order to build index pointers.
        Enables O(1) traversal of connected items without TID comparison.
        
        FOR EACH TRANSACTION T (Lines 3-17):
        1. Calculate TU(T)
        2. Revise T by total_order (only promising items)
        3. Process items from LAST→FIRST:
           - uo(i,T) = u(i,T) / TU(T)
           - ruo accumulates from tail
           - Store next_idx pointer to enable chain following
        """
        """Algorithm 2: Construct Global Utility Occupancy Indexed Lists"""
        print("  [2/3] Second database scan: constructing GUO-ILs...")
        
        for tid, transaction in self.transactions_dict.items():
            # Calculate transaction utility
            tu = self.calculate_transaction_utility(transaction)
            if tu == 0:
                continue
            
            # Filter and sort transaction by total order
            revised_trans = [(item, qty) for item, qty in transaction 
                        if item in self.item_to_idx]
            revised_trans.sort(key=lambda x: self.item_to_idx[x[0]])
            
            # Process transaction in REVERSE order (Algorithm 2, Lines 6-17)
            ruo = 0.0
            next_item = None
            next_idx = None
            
            for i in range(len(revised_trans) - 1, -1, -1):
                item, quantity = revised_trans[i]
                
                # Calculate utility occupancy (Line 11)
                u_item = self.calculate_utility(item, quantity)
                uo = u_item / tu
                
                # Create or get GUO-IL for this item
                if item not in self.guo_ils:
                    self.guo_ils[item] = IndexedList(item)
                
                # Create quadruple entry (Line 12)
                entry = QuadrupleEntry(next_item, uo, ruo, next_idx)
                self.guo_ils[item].add_entry(entry)
                
                # Update for next iteration (Lines 13-15)
                ruo = ruo + uo  # Accumulate remaining utility occupancy
                next_item = item
                next_idx = len(self.guo_ils[item].entries) - 1
        
        print(f"      Constructed {len(self.guo_ils)} GUO-ILs")
    
    def calculate_lubuo(self, indexed_list: IndexedList) -> float:
        """
        Calculate Loose Upper-Bound Utility Occupancy (Pruning Strategy 1).
        
        FORMULA (Def. 11, Eq. 12):
        LUBUO(X) = (∑uo(X) + ∑ruo(X)) / minsup
        
        PURPOSE:
        Quick conservative upper bound. If LUBUO(X) < β, no extensions possible.
        """
        """Calculate loose upper bound utility occupancy"""
        if self.min_sup == 0:
            return 0.0
        return (indexed_list.sum_uo + indexed_list.sum_ruo) / self.min_sup
    
    def calculate_ubuo(self, indexed_list: IndexedList) -> float:
        """
        Calculate Tight Upper-Bound Utility Occupancy (Pruning Strategy 2).
        
        FORMULA (Def. 9, Eq. 11):
        UBUO(X) = Σ(top_K(uo(e) + ruo(e) for e∈entries(X))) / minsup
        where K = minsup
        
        PURPOSE:
        More precise bound using top-K largest (uo+ruo) values.
        """
        """Calculate upper bound utility occupancy"""
        if self.min_sup == 0:
            return 0.0
        
        # Collect uo + ruo for each entry
        values = [entry.uo + entry.ruo for entry in indexed_list.entries]
        
        # Get top min_sup values
        if len(values) < self.min_sup:
            top_values = values
        else:
            top_values = heapq.nlargest(self.min_sup, values)
        
        return sum(top_values) / self.min_sup
    
    def mine_patterns(self, prefix: tuple, cuo_set: Dict[str, IndexedList], 
                     base_list: IndexedList = None) -> List[Tuple[tuple, float, int]]:
        """
        Algorithm 3: Search procedure - recursive DFS pattern mining.
        
        PRUNING STRATEGIES (Lines 3-9):
        1. support(P) < minsup → discard
        2. uo(P) ≥ β → output HUO-pattern  
        3. LUBUO(P) < β → pruning
        4. UBUO(P) < β → pruning
        
        PATTERN EXTENSION (Lines 10-25):
        Use create_cuo_ils() to build next-level conditional lists via index chains.
        """
        """Algorithm 3: Recursively mine patterns using DFS"""
        results = []
        
        # Process each pattern in current level (Algorithm 3, Line 2)
        for item, cuo_il in cuo_set.items():
            current_pattern = prefix + (item,)
            support = cuo_il.get_support()
            
            # Check minimum support constraint (Line 3)
            if support < self.min_sup:
                continue
            
            # Calculate utility occupancy (Lines 4-6)
            uo_value = cuo_il.get_uo()
            
            # Check if it's a high utility occupancy pattern
            if uo_value >= self.min_uo_ratio:
                results.append((current_pattern, uo_value, support))
            
            # Pruning Strategy 1: Check loose upper bound (Line 7)
            lubuo = self.calculate_lubuo(cuo_il)
            if lubuo < self.min_uo_ratio:
                continue
            
            # Pruning Strategy 2: Check tight upper bound (Lines 8-9)
            ubuo = self.calculate_ubuo(cuo_il)
            if ubuo < self.min_uo_ratio:
                continue
            
            # Extend pattern (Lines 10-25)
            # Create CUO-ILs for next level by following index chains
            next_cuo_set = self.create_cuo_ils(cuo_il, base_list)
            
            if len(next_cuo_set) > 0:
                # Recursive mining (Lines 28-29)
                # Pass current cuo_il as base_list for next level
                results.extend(self.mine_patterns(current_pattern, next_cuo_set, cuo_il))
        
        return results
    
    def create_cuo_ils(self, prefix_il: IndexedList, 
                      base_list: IndexedList = None) -> Dict[str, IndexedList]:
        """
        Algorithm 3, Lines 11-24: Create conditional IndexedLists for pattern extension.
        
        KEY INNOVATION (Lemma 2):
        Follow index chains (next_item → next_idx) directly from QuadrupleEntry.
        NO TID-list intersection required - O(1) per connection.
        
        UTILITY OCCUPANCY CALCULATION (Lines 18-20):
        new_uo = prefix_uo + connected_uo - base_uo
        Prevents double-counting prefix utility for k≥3 patterns.
        """
        """Create conditional indexed lists for pattern extension"""
        cuo_set = {}
        
        # Create new base list for this prefix (Line 12)
        new_base_list = IndexedList(prefix_il.pattern)
        
        for entry_idx, entry in enumerate(prefix_il.entries):
            # Store prefix utility occupancy in base list
            base_entry = QuadrupleEntry(None, entry.uo, 0, None)
            new_base_list.add_entry(base_entry)
            
            # Follow the chain of connected entries (Line 13)
            next_item = entry.next_item
            next_idx = entry.next_idx
            
            # Key innovation: Direct pointer following (no TID comparison)
            while next_item is not None and next_item in self.guo_ils:
                # Get the connected entry from GUO-IL (Lines 14-16)
                target_guo = self.guo_ils[next_item]
                
                if next_idx >= len(target_guo.entries):
                    break
                    
                connected_entry = target_guo.entries[next_idx]
                
                # Create or get CUO-IL for extended pattern
                if next_item not in cuo_set:
                    cuo_set[next_item] = IndexedList(next_item)
                
                # Calculate utility occupancy (Lines 18-20)
                # Line 18: Find the entry pe of Base-list in CUO_k
                if base_list is not None and entry_idx < len(base_list.entries):
                    prefix_uo = base_list.entries[entry_idx].uo
                else:
                    prefix_uo = 0.0
                
                # Line 19: ne.uo ← e.uo + ce.uo 
                # This prevents double-counting for k >= 3 itemsets
                new_uo = entry.uo + connected_entry.uo
                new_ruo = connected_entry.ruo  # Line 20
                
                # Create new entry (Line 17)
                new_entry = QuadrupleEntry(
                    connected_entry.next_item,
                    new_uo,
                    new_ruo,
                    connected_entry.next_idx
                )
                cuo_set[next_item].add_entry(new_entry)
                
                # Move to next in chain (Line 22)
                next_item = connected_entry.next_item
                next_idx = connected_entry.next_idx
        
        return cuo_set
    
    def fit(self, transactions_dict: Dict[str, List[Tuple[str, int]]], 
            profit_table: Dict[str, float]) -> List[Tuple[tuple, int, float]]:
        """
        Execute complete HUOMIL algorithm (Algorithm 1 wrapper).
        
        INPUT FORMAT:
        transactions_dict : Dict[str, List[Tuple[str,int]]]
            {TID: [(item1, qty1), (item2, qty2), ...]}
        profit_table : Dict[str, float]
            {item: profit_per_unit}
            
        RETURNS:
        List[Tuple[tuple, int, float]]:
            [(pattern_tuple, support_count, utility_occupancy), ...]
            All patterns where support ≥ α|D| AND uo ≥ β
        """
        """Main execution method (Algorithm 1)"""
        print(f"\nStarting HUOMIL Algorithm (Alpha={self.min_sup_ratio}, Beta={self.min_uo_ratio})...")
        
        # Store data
        self.transactions_dict = transactions_dict
        self.external_utilities = profit_table
        self.min_sup = max(1, int(len(transactions_dict) * self.min_sup_ratio))
        
        # Reset state
        self.result_patterns = []
        self.item_support.clear()
        self.guo_ils.clear()
        
        # Algorithm 1: Main HUOMIL
        self.first_database_scan()
        
        # Algorithm 2: Construct GUO-ILs
        self.construct_guo_ils()
        
        # Algorithm 3: Mine patterns
        print("  [3/3] Mining patterns...")
        
        # Convert GUO-ILs to initial dictionary for mining
        initial_cuo_set = {item: guo_il for item, guo_il in self.guo_ils.items()}
        
        # Start recursive mining with empty prefix
        self.result_patterns = self.mine_patterns(tuple(), initial_cuo_set)
        
        print(f"      Found {len(self.result_patterns)} patterns")
        
        # Convert to format matching (pattern, support, uo)
        formatted_results = [
            (pattern, support, uo) 
            for pattern, uo, support in self.result_patterns
        ]
        
        return formatted_results


print("HUOMIL class loaded successfully!")

HUOMIL class loaded successfully!


## Example Run (Toy Data)


In [7]:
# Small test with toy data
print("SMALL TEST: Verifying HUOMIL implementation")

toy_transactions = {
    'T1': [('A', 2), ('B', 6), ('C', 1)],
    'T2': [('A', 1), ('D', 3), ('G', 2)],
    'T3': [('B', 1), ('D', 2), ('G', 3)],
    'T4': [('A', 1), ('C', 5), ('D', 2)],
    'T5': [('B', 4), ('D', 1), ('F', 1)],
}

toy_profits = {
    'A': 3.0, 'B': 1.0, 'C': 1.0,
    'D': 5.0, 'F': 3.0, 'G': 2.0,
}

miner = HUOMIL(min_sup_ratio=0.4, min_uo_ratio=0.3)
results = miner.fit(toy_transactions, toy_profits)

print("\nResults:")
for pattern, support, uo in results[:10]:
    print(f"  {pattern}: sup={support}, uo={uo:.4f}")

print("\nSmall test completed successfully!")

SMALL TEST: Verifying HUOMIL implementation

Starting HUOMIL Algorithm (Alpha=0.4, Beta=0.3)...
  [1/3] First database scan: calculating item support...
      Found 5 promising items (min_sup=2)
  [2/3] Second database scan: constructing GUO-ILs...
      Constructed 5 GUO-ILs
  [3/3] Mining patterns...
      Found 5 patterns

Results:
  ('B', 'D'): sup=2, uo=0.6985
  ('A', 'D'): sup=2, uo=0.7702
  ('C', 'A'): sup=2, uo=0.4915
  ('D',): sup=4, uo=0.5606
  ('G', 'D'): sup=2, uo=0.9024

Small test completed successfully!
